In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from problem_generator import Problem_Generate_Model
from langchain.schema import HumanMessage, SystemMessage
from util import random_word
from util import find_random_question
import random
import re

In [9]:
total_cost = 0
verbose = True
default_model = "gpt-4o-mini"

keyword = 'TOPIK_2_READING'
problem_type = 'READING_2_PROBLEM_TYPE_3'
random_question = find_random_question(keyword, problem_type, detail_type=1, count=1, api_key_path="./../API_KEY/topik_api_key.json")
if verbose:
    print(random_question)
random_question = random_question[0]
random_question_selector = random_question['selector']
if verbose:
    print(random_question_selector)

models = [default_model]
problem_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.1, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)
selector = []
for select in random_question_selector:
    if verbose:
        print(f"selector: {select}")
    problem_generate_model.request_models_responses(
        [
            SystemMessage(content="주어지는 단어와 동일한 뜻을 지닌 다른 단어 1개를 제시해줘."),
            HumanMessage(content="바꿀 단어: " + select + " 동의어: "),
        ]
    )
    word_responses = problem_generate_model.get_model_responses()
    word = word_responses[0][1]
    def clean_text(text):
        pattern = r':\s*(.*)$'
        match = re.search(pattern, text)
        if match:
            return match.group(1).strip()
        text = re.sub(r'\n', ' ', text)  # 개행 문자를 공백으로 대체
        return text.strip()
    word = clean_text(word)
    total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', word_responses[0][2]).group(1))
    selector.append(word)
    
    if select == random_question['answer']:
        random_question['answer'] = word
    
if verbose:
    print(selector)

result = {
    "example": random_question['example'],
    "selector": selector,
    "answer": random_question['answer'],
    "eval_answer": random_question['answer'],
    "eval_explain": "",
    "total_cost": total_cost,
}

print(result)

[{'example': 'https://topikkorea.s3.amazonaws.com/topik_image/6e94c5be-23ef-11ef-a157-e5a43cd400fd.png', 'selector': ['우체국', '여행사', '편의점', '빨래방'], 'answer': '빨래방'}]
['우체국', '여행사', '편의점', '빨래방']
selector: 우체국
gpt-4o-mini | 
우편소포

OpenAI Tokens Used: 53
    Prompt Tokens: 49
    Completion Tokens: 4
Successful Requests: 1
Total Cost (USD): $4e-05
Total Cost (KRW): ₩0.054
--------------------------------------------------------------------------------------------------
selector: 여행사
gpt-4o-mini | 
여행사 동의어: 여행업체

OpenAI Tokens Used: 58
    Prompt Tokens: 48
    Completion Tokens: 10
Successful Requests: 1
Total Cost (USD): $4e-05
Total Cost (KRW): ₩0.054
--------------------------------------------------------------------------------------------------
selector: 편의점
gpt-4o-mini | 
편의점의 동의어: 슈퍼마켓

OpenAI Tokens Used: 62
    Prompt Tokens: 49
    Completion Tokens: 13
Successful Requests: 1
Total Cost (USD): $5e-05
Total Cost (KRW): ₩0.0675
----------------------------------------------------